# Find the best sample position based on a fit of the total diffraction intensity

Use the total diffracted signal on the pilatus detector to determine the optimal motor position for a given motor scan. 

In [ ]:
%matplotlib widget
import h5py
import numpy as np
import matplotlib.pyplot as plt
import DanMAX as DM

In [ ]:
#fname = '/data/visitors/danmax/PROPOSAL/VISIT/raw/SAMPLE/scan-XXXX.h5'
fname = DM.getLatestScan(scan_type='dscan')
#fname = DM.findScan()

# get the motor name from the scan type
motor = DM.getScanType(fname).split()[1]
# read data form the .h5 file
with h5py.File(fname) as fh:
    im = fh['/entry/instrument/pilatus/data'][:].astype(float)
    x = fh[f'/entry/instrument/{motor}/value'][:]

DM.getHottestPixel(fname)
# set masked pixels to nan
im[im<0]=np.nan
# mean detector intensity
I = np.nanmean(im,axis=(1,2))
I = (I-I.min())/(I.max()-I.min())

# performe Gaussian single-peak fit
amp,pos,fwhm,bgr,y_calc = DM.singlePeakFit(x,I)

print(f'Fitted maxima at {motor}: {pos:.4f} mm')
print(f'Closes measured  {motor}: {x[np.argmin(np.abs(x-pos))]:.4f} mm')

# plot the observed and fitted points
plt.figure(dpi=150)
plt.title(DM.getScan_id(fname))
plt.plot(x,I,'k.',ms=2,label='Mean intensity')
plt.plot(x,y_calc,'r-',lw=1,label='Gaussian fit')
plt.axvline(pos,color='k',linestyle=':',lw=1)
plt.annotate(f'{pos:.3f}',(pos,1),color='r')
plt.xlabel(f'{motor} position (mm)')
plt.ylabel('Intensity')
plt.legend()